In [1]:
%run moon_code
%run xgb_modelfit

In [2]:
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

In [139]:
with open('raw_data_20181008.pickle', 'rb') as handle:
    raw_data = pickle.load(handle)
with open('ks11_20181008.pickle', 'rb') as handle:
    ks11 = pickle.load(handle)

In [140]:
nan_li = ['BGF', 'BGF리테일', 'HDC현대산업개발', 'LIG넥스원', 'SK케미칼', '넷마블', '두산밥캣',
         '만도', '삼성물산', '삼성바이오로직스', '삼성에스디에스', '오리온', '이노션', '종근당',
         '코스맥스', '쿠쿠홀딩스', '현대로템', '현대중공업지주', '효성중공업', '동아에스티']
for stock in nan_li:
    del raw_data[stock]

In [141]:
def get_multi_index(col_name):
    return pd.MultiIndex.from_product([[col_name],list(raw_data.keys())])
def add_feature(q, col_name, expr):
    ind = get_multi_index(col_name)
    try:
        df = pd.DataFrame(expr.values, columns=ind, index=q.index)
    except AttributeError:
        df = pd.DataFrame(expr, columns=ind, index=q.index)
    q = pd.concat((q, df), axis=1)
    return q

In [142]:
data = {}
for stock, df in raw_data.items():
    dff = df.copy()
    dff.columns = pd.MultiIndex.from_arrays([[stock]*6, df.columns])
    data[stock] = dff

In [205]:
with open('raw_data_20181023.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [143]:
p = pd.DataFrame()
for stock, df in data.items():
    p = pd.concat((p, df), axis=1)

In [144]:
q = p.T.swaplevel().sortlevel().T

In [145]:
q.head()

Adj Close                                          \
                  AK홀딩스 BNK금융지주       CJ        CJ CGV   CJ대한통운   
Date                                                              
2010-01-04  5778.959961     NaN  62200.0  19882.162109  54500.0   
2010-01-05  5708.910156     NaN  63200.0  19976.390625  54400.0   
2010-01-06  5708.910156     NaN  63200.0  20353.304688  56400.0   
2010-01-07  5761.450195     NaN  64100.0  20494.646484  56600.0   
2010-01-08  5796.470215     NaN  67400.0  21484.044922  57600.0   

                                                                             \
                   CJ제일제당        DB손해보험   DB하이텍           GKL            GS   
Date                                                                          
2010-01-04  193995.484375  27523.703125  7500.0  15079.077148  27447.957031   
2010-01-05  206178.765625  27040.828125  7550.0  15842.575195  27569.951172   
2010-01-06  206647.375000  28529.683594  7410.0  15804.404297  27854.591797   
2010-01-07  204304.437500  28690.644531  7460.0  15613.526367  28139.236328   
2010-01-08  204304.437500  28408.968750  7510.0  17331.396484  28017.250000   

              ...    Volume                                                    \
              ...      현대위아       현대제철      현대중공업        현대차       현대해상 현대홈쇼핑   
Date          ...                                                               
2010-01-04    ...       NaN  1422792.0   327580.0  1602697.0  1317347.0   NaN   
2010-01-05    ...       NaN  1946501.0   362845.0  3045198.0  1057926.0   NaN   
2010-01-06    ...       NaN  1238290.0   694478.0  1681152.0  1292660.0   NaN   
2010-01-07    ...       NaN  1290509.0  2941116.0  1736811.0   950092.0   NaN   
2010-01-08    ...       NaN   997631.0  1834213.0  2175189.0   951793.0   NaN   

                                                      
                호텔신라         효성         후성       휴켐스  
Date                                                  
2010-01-04  494447.0   590567.0   784870.0  344720.0  
2010-01-05  591959.0  1716465.0  1010749.0  671702.0  
2010-01-06  511434.0  2139316.0  1064106.0  276467.0  
2010-01-07  366978.0  1056023.0  2206727.0  232912.0  
2010-01-08  343782.0  1409472.0   996558.0  181563.0  

[5 rows x 1092 columns]

In [146]:
import time
start = time.time()
q = add_feature(q, 'pct_change', q.Close.pct_change())
for i in range(3, 31, 2):
    q = add_feature(q, "close_ma_{}".format(i), q.Close.rolling(window=i).mean())
    q = add_feature(q, "volume_ma_{}".format(i), q.Volume.rolling(window=i).mean())
for i in [7, 21, 30, 60, 90, 91, 100, 120]:
    q = add_feature(q, "his_vol_{}".format(i), q['pct_change'].rolling(window=i).std()*(252**0.5))
for i in [1, 3, 5, 7, 10, 12, 14, 16]:
    q = add_feature(q, 'returnsClosePrevRaw{}'.format(i), np.log(q.Open / q.Close.shift(1)))
    q = add_feature(q, 'returnsOpenPrevRaw{}'.format(i), np.log(q.Open / q.Open.shift(1)))
for i in [1, 3, 5, 7, 10]:
    q = add_feature(q, 'Y3_{}'.format(i), 
                    np.where(q.Close.rolling(5).mean() - q.Close.shift(1).rolling(10).mean() >=0, 1, 0))
    for shift in [1, 2, 3, 4, 5]:
        q = add_feature(q, 'sh{}_Y3_{}'.format(shift, i), 
                    q['Y3_{}'.format(i)].shift(-1*shift))
elapse = time.time() - start

In [147]:
print(elapse)

23.961437940597534


In [148]:
# Adjust Time Period
q2 = q.loc['2013-07-01':'2018-07-01'].copy()

In [149]:
q2.head()

Adj Close                                                  \
                   AK홀딩스       BNK금융지주        CJ        CJ CGV    CJ대한통운   
Date                                                                       
2013-07-01  26722.099609  12409.956055  117000.0  49895.128906   98200.0   
2013-07-02  27397.300781  12705.503906  119500.0  49895.128906  101500.0   
2013-07-03  27686.699219  12578.802734  118000.0  49506.082031   98000.0   
2013-07-04  27976.199219  12832.117188  117000.0  50381.433594   98000.0   
2013-07-05  28651.400391  12958.727539  118000.0  48144.421875   99100.0   

                                                                             \
                   CJ제일제당        DB손해보험   DB하이텍           GKL            GS   
Date                                                                          
2013-07-01  246251.718750  41898.656250  6860.0  28783.619141  44063.058594   
2013-07-02  260737.062500  42340.625000  6700.0  29079.919922  43802.839844   
2013-07-03  253977.250000  42650.003906  6450.0  27894.708984  43152.300781   
2013-07-04  256874.328125  42031.250000  6520.0  28402.658203  43976.320313   
2013-07-05  255908.609375  43224.566406  7490.0  28402.658203  44670.222656   

           ...  sh5_Y3_10                                                 
           ...       현대위아 현대제철 현대중공업  현대차 현대해상 현대홈쇼핑 호텔신라   효성   후성  휴켐스  
Date       ...                                                            
2013-07-01 ...        0.0  0.0   1.0  1.0  1.0   1.0  1.0  1.0  1.0  0.0  
2013-07-02 ...        1.0  0.0   1.0  0.0  1.0   1.0  1.0  1.0  1.0  1.0  
2013-07-03 ...        0.0  0.0   1.0  0.0  1.0   1.0  1.0  1.0  1.0  1.0  
2013-07-04 ...        0.0  1.0   1.0  0.0  1.0   1.0  0.0  1.0  1.0  1.0  
2013-07-05 ...        0.0  1.0   1.0  0.0  1.0   1.0  0.0  1.0  1.0  1.0  

[5 rows x 16198 columns]

In [150]:
q2.isnull().sum().sum()

0

In [151]:
total_col = list(q2.columns.get_level_values(0).unique())
target_li = [col for col in total_col if col.find('Y') != -1]
X_li = [col for col in total_col if col.find('Y') == -1]

In [152]:
scl_q = q2.copy()

In [153]:
scl_col = pd.MultiIndex.from_product([['scl_'+i for i in list(scl_q.columns.get_level_values(0).unique())], 
                            list(scl_q.columns.get_level_values(1).unique())])
scl_q.columns = scl_col

In [163]:
scl_X_li = list(scl_q.columns.get_level_values(0).unique())
scl_li = scl_X_li.copy()
del_li = []
for ix, i in enumerate(scl_X_li):
    if (i.find('returns') != -1) or (i.find('his') != -1) or (i.find('pct') != -1):
        scl_li[ix] = scl_X_li[ix][4:]
    elif (i.find('Y') != -1):
        del_li.append(i)
scl_X_li = sorted(list(set(scl_X_li) - set(del_li)))
scl_li = sorted(list(set(scl_li) - set(del_li)))

In [164]:
origin_X_li = [i for i in X_li if i.find('returns') != 0]
origin_scl_X_li = [i for i in scl_li if i.find('returns') != 0]

In [169]:
scl_q = scl_q[scl_X_li]

In [219]:
train1, test1 = train_test_split(q2, 0.8, dtrain=True)
train2, test2 = train_test_split(scl_q, 0.8, dtrain=True)

In [220]:
def apply_standard_scale(train, test, predictors):
    std_scaler = StandardScaler()
    std_scaler.fit(train[predictors].values)
    sub_train = std_scaler.transform(train[predictors].values)
    sub_test = std_scaler.transform(test[predictors].values)
    sub_train = pd.DataFrame(sub_train, columns = train.columns, index = train.index)
    sub_test = pd.DataFrame(sub_test, columns = test.columns, index = test.index)
    return sub_train, sub_test

In [221]:
train2, test2 = apply_standard_scale(train2, test2, scl_X_li)

In [222]:
train, test = pd.concat((train1, train2), axis=1), pd.concat((test1, test2), axis=1)

In [224]:
store = (train, test, target_li, X_li, scl_X_li, scl_li)

with open("store_학습준비파일_20181023.pickle", 'wb') as handle:
    pickle.dump(store, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [225]:
with open("store_학습준비파일_20181023.pickle", 'rb') as handle:
    store = pickle.load(handle)

In [227]:
store[3]

['Adj Close',
 'Close',
 'High',
 'Low',
 'Open',
 'Volume',
 'pct_change',
 'close_ma_3',
 'volume_ma_3',
 'close_ma_5',
 'volume_ma_5',
 'close_ma_7',
 'volume_ma_7',
 'close_ma_9',
 'volume_ma_9',
 'close_ma_11',
 'volume_ma_11',
 'close_ma_13',
 'volume_ma_13',
 'close_ma_15',
 'volume_ma_15',
 'close_ma_17',
 'volume_ma_17',
 'close_ma_19',
 'volume_ma_19',
 'close_ma_21',
 'volume_ma_21',
 'close_ma_23',
 'volume_ma_23',
 'close_ma_25',
 'volume_ma_25',
 'close_ma_27',
 'volume_ma_27',
 'close_ma_29',
 'volume_ma_29',
 'his_vol_7',
 'his_vol_21',
 'his_vol_30',
 'his_vol_60',
 'his_vol_90',
 'his_vol_91',
 'his_vol_100',
 'his_vol_120',
 'returnsClosePrevRaw1',
 'returnsOpenPrevRaw1',
 'returnsClosePrevRaw3',
 'returnsOpenPrevRaw3',
 'returnsClosePrevRaw5',
 'returnsOpenPrevRaw5',
 'returnsClosePrevRaw7',
 'returnsOpenPrevRaw7',
 'returnsClosePrevRaw10',
 'returnsOpenPrevRaw10',
 'returnsClosePrevRaw12',
 'returnsOpenPrevRaw12',
 'returnsClosePrevRaw14',
 'returnsOpenPrevRaw14',


In [209]:
train = train.T.swaplevel().sortlevel().T
test = test.T.swaplevel().sortlevel().T

In [211]:
train_df = train.copy()
test_df = test.copy()

In [218]:
del_li

['scl_Y3_1',
 'scl_sh1_Y3_1',
 'scl_sh2_Y3_1',
 'scl_sh3_Y3_1',
 'scl_sh4_Y3_1',
 'scl_sh5_Y3_1',
 'scl_Y3_3',
 'scl_sh1_Y3_3',
 'scl_sh2_Y3_3',
 'scl_sh3_Y3_3',
 'scl_sh4_Y3_3',
 'scl_sh5_Y3_3',
 'scl_Y3_5',
 'scl_sh1_Y3_5',
 'scl_sh2_Y3_5',
 'scl_sh3_Y3_5',
 'scl_sh4_Y3_5',
 'scl_sh5_Y3_5',
 'scl_Y3_7',
 'scl_sh1_Y3_7',
 'scl_sh2_Y3_7',
 'scl_sh3_Y3_7',
 'scl_sh4_Y3_7',
 'scl_sh5_Y3_7',
 'scl_Y3_10',
 'scl_sh1_Y3_10',
 'scl_sh2_Y3_10',
 'scl_sh3_Y3_10',
 'scl_sh4_Y3_10',
 'scl_sh5_Y3_10']

# 부록

In [191]:
test = np.random.randint(1,100,(6,5))
std = StandardScaler()
std.fit(test)
print(std.transform(test))

[[-0.93078392 -1.74493187 -1.2367011   1.33134267 -0.19785804]
 [-1.06270605  0.58618805  0.53996809 -0.27519374 -0.81969758]
 [ 0.12459312 -0.92699505 -0.50513144  0.48344846 -0.59357411]
 [ 0.56433356  0.29991016 -0.97542622 -1.52472205 -0.76316672]
 [-0.53501753  0.70887857  0.53996809  0.88508256  2.06337667]
 [ 1.83958082  1.07695014  1.63732258 -0.8999579   0.31091977]]


In [192]:
test

array([[ 4,  3, 31, 98, 53],
       [ 1, 60, 65, 62, 42],
       [28, 23, 45, 79, 46],
       [38, 53, 36, 34, 43],
       [13, 63, 65, 88, 93],
       [67, 72, 86, 48, 62]])

In [ ]:
t